In [ ]:
import os
import json
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets, transforms, models

from tqdm.notebook import tqdm
from baseline_model import (
    set_seed,
    get_flowers_dataloaders,
    create_baseline_model,
    train_epoch,
    validate,
)
import pytorch_lightning as pl

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_seed()
train_loader, val_loader, test_loader = get_flowers_dataloaders(batch_size=32)

# Different Depths

In [ ]:
class EarlyStopper:
    def __init__(self, patience=10, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def __call__(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
def train_model(num_epochs = 30):
    pass

In [ ]:



config = {
    "num_epochs": 30,
    "batch_size": 32,
    "learning_rate": 0.001,
    "weight_decay": 1e-4,
    "scheduler_step_size": 10,
    "scheduler_gamma": 0.1,
}

# test different depth
resnet_depth = []
resnet_test_acc = []
resnet_overfit_gap = []
for depth, model_name in [
    (18, "resnet18"),
    (34, "resnet34"),
    (50, "resnet50"),
    (101, "resnet101"),
    (152, "resnet152"),
]:
    model = create_baseline_model(
        num_classes=102, pretrained=True, model_name=model_name
    )
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        model.parameters(),
        lr=config["learning_rate"],
        weight_decay=config["weight_decay"],
    )

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.StepLR(
        optimizer,
        step_size=config["scheduler_step_size"],
        gamma=config["scheduler_gamma"],
    )

In [ ]:
# def create_vit_model(num_classes=102):

# Reduce Parameters

# Deformable Convolutions